In [4]:
#Import Dependencies
import csv
import pandas as pd
import requests
import json
#from config import API_KEY, gkey
from pprint import pprint
import gmaps

In [5]:
#Import Beer Review Data
beer_data=pd.read_csv('beer_reviews.csv', encoding = "ISO-8859-1")

#Preview the data
beer_data.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [6]:
#Group the data by beer name, brewery, and beer type
beer = beer_data.groupby(["beer_name","brewery_name","beer_style"])

In [12]:
#Create a data frame for average ratings based on individual beers
beer_df = pd.DataFrame(beer.mean())
ind_beer_rating_df = beer_df.reset_index(["brewery_name", "beer_style"])

#Remove brewery_id, review_time, and beer_id
ind_beer_rating_df = ind_beer_rating_df.drop(columns=(["brewery_id", "review_time", "beer_beerid"]))

#Display the data frame
ind_beer_rating_df

,brewery_name,beer_style,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
beer_name,,,,,,,,
! (Old Ale),NÃ¤rke Kulturbryggeri AB,Old Ale,4.000000,4.500000,4.000000,4.000000,4.500000,8.20
"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,4.000000,4.000000,4.000000,4.000000,4.400000,10.00
"""100"" Pale Ale",Aviator Brewing Company,American IPA,4.000000,4.000000,3.500000,4.000000,4.000000,6.60
"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,4.000000,3.500000,4.250000,4.750000,4.500000,9.00
"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,3.000000,3.000000,3.000000,3.166667,3.000000,4.80
"""4"" Horse Oatmeal Stout",Thoroughbreds Grill & Brewing,Oatmeal Stout,4.000000,3.500000,4.000000,3.250000,3.250000,NaN
"""400"" Ale",Williamsburg AleWerks,American Brown Ale,3.333333,3.833333,3.916667,3.750000,3.444444,NaN
"""50"" Golden Anniversary Lager",Mia & Pia's Pizzeria And Brewhouse,American Pale Lager,4.000000,3.500000,5.000000,3.500000,3.500000,4.00
"""76"" Anniversary Ale",Bootlegger's Brewery,Old Ale,4.000000,4.166667,3.666667,4.000000,4.166667,7.60


In [15]:
#Create a dataframe of Breweries
breweries = list(beer_data['brewery_name'].unique())
breweries_df = pd.DataFrame(breweries)

#Remove special brewery names with weird characters
clean_brew_df = breweries_df[~breweries_df[0].astype(str).str.contains('[^a-zA-Z0-9 ]', regex=True)]
clean_brew_df = clean_brew_df.reset_index()
clean_brew_df = clean_brew_df.drop(columns='index')
clean_brew_df = clean_brew_df.rename(columns=({0: "Brewery"}))

#Preview the data
clean_brew_df.head()
clean_list = list(clean_brew_df['Brewery'])
clean_list

['Vecchio Birraio',
 'Caldera Brewing Company',
 'Broad Ripple Brew Pub',
 'Moon River Brewing Company',
 'Kirner Bier',
 'Conwy Brewery Ltd',
 'Pacific Coast Brewing Company',
 'City Grille and Brewhaus',
 'San Francisco Brewing Company',
 'Yazoo Brewing Company',
 '7 Seas Brewery and Taproom',
 'Calapooia Brewing Company',
 'Brouwerij Van Viven',
 'Destiny Brewing Company',
 'Hokusetsu Brewery',
 'Bernard Family Brewery',
 'Isle Of Skye Brewery',
 'Brauhaus Lemke',
 'Founders Brewing Company',
 'The Stable',
 'Finger Lakes Beer Company',
 'Rustic Rail Grill And Brewhouse',
 'Craggie Brewing Company',
 'Brauerei Neder',
 'Brew Eyed',
 'NoDa Brewing Company',
 'White Bluffs Brewing Company',
 'Golden State Brewing Company',
 'Growler Craft Brewery',
 'Great South Bay Brewery',
 'Mariposa Brewing Company',
 '4th Street Brewing Company',
 'Yeastie Boys',
 'Stadtbrauerei Arnstadt',
 'Asmara Breweries',
 'Voronezhsky Brewery',
 'Ohio Brewing Company',
 'Crescent City Brewhouse',
 'Pivovar 